# Reinforcement Learning Solution to the Towers of Hanoi Puzzle

Michael Lynn

For this assignment, you will use reinforcement learning to solve the [Towers of Hanoi](https://en.wikipedia.org/wiki/Tower_of_Hanoi) puzzle.  

To accomplish this, you must modify the code discussed in lecture for learning to play Tic-Tac-Toe.  Modify the code  so that it learns to solve the three-disk, three-peg
Towers of Hanoi Puzzle.  In some ways, this will be simpler than the
Tic-Tac-Toe code.  

Steps required to do this include the following:

  - Represent the state, and use it as a tuple as a key to the Q dictionary.
  - Make sure only valid moves are tried from each state.
  - Assign reinforcement of $1$ to each move, even for the move that results in the goal state.

Make a plot of the number of steps required to reach the goal for each
trial.  Each trial starts from the same initial state.  Decay epsilon
as in the Tic-Tac-Toe code.

## Requirements

First, how should we represent the state of this puzzle?  We need to keep track of which disks are on which pegs. Name the disks 1, 2, and 3, with 1 being the smallest disk and 3 being the largest. The set of disks on a peg can be represented as a list of integers.  Then the state can be a list of three lists.

For example, the starting state with all disks being on the left peg would be `[[1, 2, 3], [], []]`.  After moving disk 1 to peg 2, we have `[[2, 3], [1], []]`.

To represent that move we just made, we can use a list of two peg numbers, like `[1, 2]`, representing a move of the top disk on peg 1 to peg 2.

Now on to some functions. Define at least the following functions. Examples showing required output appear below.

   - `printState(state)`: prints the state in the form shown below
   - `validMoves(state)`: returns list of moves that are valid from `state`
   - `makeMove(state, move)`: returns new (copy of) state after move has been applied.
   - `trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF)`: train the Q function for number of repetitions, decaying epsilon at start of each repetition. Returns Q and list or array of number of steps to reach goal for each repetition.
   - `testQ(Q, maxSteps, validMovesF, makeMoveF)`: without updating Q, use Q to find greedy action each step until goal is found. Return path of states.

A function that you might choose to implement is

   - `stateMoveTuple(state, move)`: returns tuple of state and move.  
    
This is useful for converting state and move to a key to be used for the Q dictionary.

Show the code and results for testing each function.  Then experiment with various values of `nRepetitions`, `learningRate`, and `epsilonDecayFactor` to find values that work reasonably well, meaning that eventually the minimum solution path of seven steps is found consistently.

Make a plot of the number of steps in the solution path versus number of repetitions. The plot should clearly show the number of steps in the solution path eventually reaching the minimum of seven steps, though the decrease will not be monotonic.  Also plot a horizontal, dashed line at 7 to show the optimal path length.

Add markdown cells in which you describe the Q learning algorithm and your implementation of Q learning as applied to the Towers of Hanoi problem.  Use at least 15 sentences, in one or more markdown cells.

In [281]:
import numpy as np
import copy

def printState(myState):
    rowOne = 0
    rowTwo = 0
    rowThree = 0
    colOne = 0
    colTwo = 1
    colThree = 2
    # interate through 3x3 matrix
    for i in range(3,0,-1):
        temp = []
        if len(myState[colOne]) >= i and len(myState[colOne]) > 0:           
            print(myState[colOne][rowOne], end="")
            print(" ", end="")
            rowOne += 1
        else:
            print(" ", end="")
        if len(myState[colTwo]) >= i and len(myState[colTwo]) > 0:
            print(myState[colTwo][rowTwo], end="")
            print(" ", end="")
            rowTwo += 1
        else:
            print(" ", end="")
        if len(myState[colThree]) >= i and len(myState[colThree]) > 0:
            print(myState[colThree][rowThree], end="")
            print(" ", end="")
            rowThree += 1
        else:
            print(" ", end="")
        print()
    print("------")
    
def stateMoveTuple(myState, myMove):
    colOne = tuple(myState[0])
  #  print(colOne);
    colTwo = tuple(myState[1])
  #  print(colTwo);
    colthree = tuple(myState[2])
  #  print(colthree);
    return ((colOne,colTwo,colthree), tuple(myMove))

def makeMove(myState, myMove):
    # offset disk and tower (1,2) to (0-1)  3x3 matrix

    offSetDisc = myMove[0]-1
    offSetTower = myMove[1]-1

    targetDisc = myState[offSetDisc][0]
    targetTower = offSetTower
    
    # remove targetDisc 
    myState[offSetDisc] = myState[offSetDisc][1:]
    
    #insert targetDisc into targetTower
    myState[targetTower].insert(0, targetDisc)
    return myState

def validMoves(myState):
    t = 4;
    #tower 1 state 
    colOne = myState[0]
    #tower 2 state 
    colTwo = myState[1]
    #tower 3 state 
    colThree = myState[2]

    myMoves = []
    
    topDiskTower1 = t
    
    topDiskTower2 = t
    
    topDiskTower3 = t

    # grab top disk is tower is not empty
    if len(colOne) > 0:
        topDiskTower1 = colOne[0]
        #print(topDiskTower1)
    if len(colTwo) > 0:
        topDiskTower2 = colTwo[0]
       # print(topDiskTower2)
    if len(colThree) > 0:
        topDiskTower3 = colThree[0]
       # print(topDiskTower3)

    if topDiskTower1 < topDiskTower2:
        myMoves.append([1,2])
    if topDiskTower1 < topDiskTower3:
        myMoves.append([1,3])
    if topDiskTower2 < topDiskTower1:
        myMoves.append([2,1])
    if topDiskTower2 < topDiskTower3:
        myMoves.append([2,3])
    if topDiskTower3 < topDiskTower1:
        myMoves.append([3,1])
    if topDiskTower3 < topDiskTower2:
        myMoves.append([3,2])

    return myMoves

def epsilonGreedy(epsilon, Q, myState):
    myMoves = validMoves(myState)
    #print("mymoves ", myMoves)
    # generate random uniform default interval 0-1 floating points
    randomMove = np.random.uniform()
    if randomMove < epsilon:
        #print("explore ", myMoves[ np.random.choice(len(myMoves))])
        # return random number based on elements in target tower
        # explore a bit
        return myMoves[ np.random.choice(len(myMoves))]
    else:
        #print("move greedy Q", Q)
        # exploit what we already know
        Qs = np.array([Q.get(stateMoveTuple(myState, m), 1) for m in myMoves])
        return myMoves[ np.argmax(Qs) ]

def trainQ(trainCount, learningRate, DecayRate, validMovesF, makeEpsilonMove):
    Q = {}
    myEpsilon = 1.0
    steps = []

    for num in range(trainCount):
        myEpsilon *= DecayRate
        #print("decay ", myEpsilon)
        myStepCount = 0
        myState = [[1,2,3],[],[]]
        oldmove = []
        myOldstate = []
        done = False

        while not done:
            myStepCount += 1

            # return explore or exploit move
            epsilionMove = epsilonGreedy(myEpsilon, Q, myState)
            #print("eplsion return ", epsilionMove)
            stateNew = []
            stateNew.append(copy.copy(myState[0]))
            stateNew.append(copy.copy(myState[1]))
            stateNew.append(copy.copy(myState[2]))

            stateNew = makeEpsilonMove(stateNew, epsilionMove)

            if stateMoveTuple(myState, epsilionMove) not in Q:
                # add new state and assoicated move to Q and init to 1
                #print("new state ", stateMoveTuple(myState, epsilionMove) )
                Q[stateMoveTuple(myState, epsilionMove)] = 1
            else:
                if myStepCount > 1:
                    oldStateTuple = stateMoveTuple(myOldstate, oldmove)
                    #print("this state already exist ", oldStateTuple, "learning ", Q[oldStateTuple])
                    Q[oldStateTuple] += learningRate * (-1 + Q[stateMoveTuple(myState, epsilionMove)] - Q[oldStateTuple])
            myOldstate, oldmove = myState, epsilionMove
            myState = stateNew
            if myState == [[],[],[1,2,3]]:
                done = True
                Q[stateMoveTuple(myState,epsilionMove)] = -1

        steps.append(myStepCount)
    return Q, steps

def testQ(Q, myMaxSteps, pewteeweet, myMakeMove):
    myState = [[1,2,3],[],[]]
    allStates = []
    for i in range(myMaxSteps):
        myMove = greedy(Q, myState)
        stateNew = []
        stateNew.append(copy.copy(myState[0]))
        stateNew.append(copy.copy(myState[1]))
        stateNew.append(copy.copy(myState[2]))
        stateNew = myMakeMove(stateNew, myMove)
        if myState == [[],[],[1,2,3]]:
            break
        allStates.append(stateNew)
        myState = stateNew
    return allStates

def greedy(Q, myState):
    myMoves = validMoves(myState)
    myQs = np.array([Q.get(stateMoveTuple(myState, m), 1) for m in myMoves])
    return myMoves[ np.argmax(myQs) ]


# Discussion

describe the trainQ and testQ functions. Q based learning uses a Numerical based reward, we attach a value reward to each step.  We map trasitions between states with a reward. Everytime we get a duplicate state the learning varible is updated. Setting the epsilion to a high or lower number increases or decreseses the amount of exploing and exploiting that happens during the trainQ phase. TestQ uses the Q table produced and takes the greedy approch to get optimal solution 2^n − 1.  We can represented as a graph, kinda like a shortest path algorthm.

# Examples

In [282]:
state = [[1, 2, 3], [], []]
printState(state)

1   
2   
3   
------


In [283]:
move =[1, 2]

stateMoveTuple(state, move)

(((1, 2, 3), (), ()), (1, 2))

In [284]:
newstate = makeMove(state, move)
newstate

[[2, 3], [1], []]

In [285]:
printState(newstate)

   
2   
3 1  
------


In [286]:
Q, stepsToGoal = trainQ(50, 0.5, 0.7, validMoves, makeMove)

In [287]:
stepsToGoal

[85,
 78,
 147,
 29,
 37,
 16,
 7,
 14,
 41,
 11,
 8,
 13,
 33,
 21,
 17,
 10,
 12,
 42,
 7,
 7,
 9,
 7,
 9,
 25,
 7,
 7,
 8,
 11,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7]

In [288]:
path = testQ(Q, 20, validMoves, makeMove)

In [289]:
path

[[[2, 3], [], [1]],
 [[3], [2], [1]],
 [[3], [1, 2], []],
 [[], [1, 2], [3]],
 [[1], [2], [3]],
 [[1], [], [2, 3]],
 [[], [], [1, 2, 3]]]

In [290]:
for s in path:
    printState(s)
    print()

   
2   
3  1 
------

   
   
3 2 1 
------

   
 1  
3 2  
------

   
 1  
 2 3 
------

   
   
1 2 3 
------

   
  2 
1  3 
------

  1 
  2 
  3 
------



## Grading

Download and extract `A4grader.py` from [A4grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A4grader.tar).

In [291]:
%run -i A5grader.py


======================= Code Execution =======================

Extracting python code from notebook named 'Lynn-A4.ipynb' and storing in notebookcode.py
Removing all statements that are not function or class defs or import statements.

Testing validMoves([[1], [2], [3]])

--- 10/10 points. Correctly returned [[1, 2], [1, 3], [2, 3]]

Testing validMoves([[], [], [1, 2, 3]])

--- 10/10 points. Correctly returned [[3, 1], [3, 2]]

Testing makeMove([[], [], [1, 2, 3]], [3, 2])

--- 10/10 points. Correctly returned [[], [1], [2, 3]]

Testing makeMove([[2], [3], [1]], [1, 2])

--- 10/10 points. Correctly returned [[], [2, 3], [1]]

Testing   Q, steps = trainQ(1000, 0.5, 0.7, validMoves, makeMove).

--- 10/10 points. Q dictionary has correct number of entries.

--- 10/10 points. The mean of the number of steps of 7.506 is correctly < 10.

Testing   path = testQ(Q, 20, validMoves, makeMove).

--- 20/20 points. Correctly returns path of length 7, which is correctly less than 10.

C:\Users\hai

## Extra Credit

Modify your code to solve the Towers of Hanoi puzzle with 4 disks instead of 3.  Name your functions

    - printState_4disk
    - validMoves_4disk
    - makeMove_4disk

Find values for number of repetitions, learning rate, and epsilon decay factor for which trainQ learns a Q function that testQ can use to find the shortest solution path.  Include the output from the successful calls to trainQ and testQ.